In [1]:
import pandas as pd
import numpy as np
import os

df_li = []
PATH = "Download/Cleaned Data/"

for file in os.listdir(PATH):
    if file.startswith('.') or 'spanish' in file.lower():
        continue
        
    df = pd.read_csv(os.path.join(PATH, file))
    df_li.append(df[:500])
    
df = pd.concat(df_li)
df

,created_utc,id,score,selftext,subreddit,title,title_language,selftext_language
0,1404687508,2a02qe,37.0,Or was this a waste of time?,StoriesAboutKevin,So are we ever gonna have any stories on Kevin?,en,en
1,1408829453,2ee903,23.0,Paging [u/NoahtheRed]! We could really use som...,StoriesAboutKevin,Come on Kevin!! OP Please Deliver!,en,en
2,1409061932,2emlgx,28.0,"I don't know about everyone else, but I know 4...",StoriesAboutKevin,"There Are Many Retarded ""Kevins""",en,en
3,1412014244,2ht4oe,41.0,"So, I was browsing Askreddit and found ""Someth...",StoriesAboutKevin,Just found out about this today,en,en
4,1414689096,2ksqpi,68.0,I was living with my older brother while atten...,StoriesAboutKevin,I am embarrassed to say that this Kevin is my ...,en,en
...,...,...,...,...,...,...,...,...
495,1251844231,9gd34,2.0,This is all just a thought. I'm not trying to...,self,I think I figured out why the south dominates ...,en,en
496,1251844989,9gd6a,0.0,If you see a particularly funny/insightful com...,self,Do you downvote comments relative to the ones ...,en,en
497,1251851288,9gdyq,0.0,"""Love it or leave it."" Do you realize how ret...",self,"Hey Neocons, love it or leave it.",en,en
498,1251852724,9ge4y,0.0,Does anyone remember these toy robots that tra...,self,pepsi can robots?,ca,en


In [2]:
def category(row):
    return 'Creepy' if row['subreddit'] in ['creepypasta', 'nosleep', 'shortscarystories'] else 'Non-creepy'

print(df['subreddit'].unique())

df['category'] = df.apply(category, axis=1)
df['category'].unique()

['StoriesAboutKevin' 'creepypasta' 'nosleep' 'confession'
 'shortscarystories' 'confessions' 'tifu' 'self']


array(['Non-creepy', 'Creepy'], dtype=object)

# Visualizing term associations

In [3]:
import spacy

nlp = spacy.load('en_core_web_lg')

In [55]:
import scattertext as st
corpus = st.CorpusFromPandas(df, 
                             category_col='category', 
                             text_col='selftext',
                             nlp=nlp).build()

In [5]:
print(list(corpus.get_scaled_f_scores_vs_background().index[:10]))

['reddit', 'kevina', 'kevin', 'subreddit', 'creepypasta', 'creepy', 'woke', 'hallway', 'facebook', 'nosleep']


In [6]:
term_freq_df = corpus.get_term_freq_df()
term_freq_df

,Non-creepy freq,Creepy freq
term,,
or,1888,1819
was,8870,11538
this,4476,3522
a,14784,13637
waste,39,14
...,...,...
even use,1,0
use reddit,1,0
matter will,1,0


In [7]:
from pprint import pprint
term_freq_df['Creepy Score'] = corpus.get_scaled_f_scores('Creepy')
pprint(list(term_freq_df.sort_values(by='Creepy Score', ascending=False).index[:20]))

['my eyes',
 'dark',
 'eyes',
 'sound',
 'the window',
 'light',
 'woods',
 'the man',
 'creepy',
 'window',
 'dream',
 'the light',
 'lights',
 'i looked',
 'mirror',
 'blood',
 'staring',
 'i saw',
 'bed',
 'stairs']


In [8]:
term_freq_df['Noncreepy Score'] = corpus.get_scaled_f_scores('Non-creepy')
pprint(list(term_freq_df.sort_values(by='Noncreepy Score', ascending=False).index[:20]))

['kevin',
 'kevin was',
 'kevina',
 "kevin 's",
 'sex',
 'teacher',
 'relationship',
 'class',
 'money',
 'the teacher',
 'hate',
 'feel like',
 'grade',
 'a kevin',
 'high school',
 '$',
 'reddit',
 'girlfriend',
 'because he',
 'school']


In [10]:
html = st.produce_scattertext_explorer(corpus,
                                       category='Creepy',
                                       category_name='Creepy',
                                       not_category_name='Non-creepy',
                                       width_in_pixels=1000)
open("Download/d3corpusfreq2.html", 'wb').write(html.encode('utf-8'))

9166167

# Visualizing Phrase Associations

In [35]:
df

,created_utc,id,score,selftext,subreddit,title,title_language,selftext_language,category
0,1404687508,2a02qe,37.0,Or was this a waste of time?,StoriesAboutKevin,So are we ever gonna have any stories on Kevin?,en,en,Non-creepy
1,1408829453,2ee903,23.0,Paging [u/NoahtheRed]! We could really use som...,StoriesAboutKevin,Come on Kevin!! OP Please Deliver!,en,en,Non-creepy
2,1409061932,2emlgx,28.0,"I don't know about everyone else, but I know 4...",StoriesAboutKevin,"There Are Many Retarded ""Kevins""",en,en,Non-creepy
3,1412014244,2ht4oe,41.0,"So, I was browsing Askreddit and found ""Someth...",StoriesAboutKevin,Just found out about this today,en,en,Non-creepy
4,1414689096,2ksqpi,68.0,I was living with my older brother while atten...,StoriesAboutKevin,I am embarrassed to say that this Kevin is my ...,en,en,Non-creepy
...,...,...,...,...,...,...,...,...,...
495,1251844231,9gd34,2.0,This is all just a thought. I'm not trying to...,self,I think I figured out why the south dominates ...,en,en,Non-creepy
496,1251844989,9gd6a,0.0,If you see a particularly funny/insightful com...,self,Do you downvote comments relative to the ones ...,en,en,Non-creepy
497,1251851288,9gdyq,0.0,"""Love it or leave it."" Do you realize how ret...",self,"Hey Neocons, love it or leave it.",en,en,Non-creepy
498,1251852724,9ge4y,0.0,Does anyone remember these toy robots that tra...,self,pepsi can robots?,ca,en,Non-creepy


In [37]:
import spacy
from scattertext import SampleCorpora, PhraseMachinePhrases, dense_rank, RankDifference, AssociationCompactor, produce_scattertext_explorer
from scattertext.CorpusFromPandas import CorpusFromPandas

corpus = (CorpusFromPandas(df,
                           category_col='category',
                           text_col='selftext',
                           feats_from_spacy_doc=PhraseMachinePhrases(),
                           nlp=spacy.load('en_core_web_lg'))
          .build().compact(AssociationCompactor(4000)))

html = produce_scattertext_explorer(corpus,
                                    category='Creepy',
                                    category_name='Creepy',
                                    not_category_name='Non-creepy',
                                    minimum_term_frequency=0,
                                    pmi_threshold_coefficient=0,
                                    transform=dense_rank,
                                    term_scorer=RankDifference(),
                                    width_in_pixels=1000)

/home/anthony/.local/share/virtualenvs/Research-Mapping-Uncanny-Valley-DSc8QBrC/lib/python3.8/site-packages/scattertext/termscoring/ScaledFScore.py:296: RuntimeWarning: invalid value encountered in true_divide
  recall = cat_word_counts * 1. / cat_word_counts.sum()


In [38]:
open("Download/test.html", 'wb').write(html.encode('utf-8'))

7339718

In [57]:
convention_df = df.assign(parse=lambda df: df['selftext'].apply(nlp))

In [58]:
convention_df

,created_utc,id,score,selftext,subreddit,title,title_language,selftext_language,category,parse
0,1404687508,2a02qe,37.0,Or was this a waste of time?,StoriesAboutKevin,So are we ever gonna have any stories on Kevin?,en,en,Non-creepy,"(Or, was, this, a, waste, of, time, ?)"
1,1408829453,2ee903,23.0,Paging [u/NoahtheRed]! We could really use som...,StoriesAboutKevin,Come on Kevin!! OP Please Deliver!,en,en,Non-creepy,"(Paging, [, u, /, NoahtheRed, ], !, We, could,..."
2,1409061932,2emlgx,28.0,"I don't know about everyone else, but I know 4...",StoriesAboutKevin,"There Are Many Retarded ""Kevins""",en,en,Non-creepy,"(I, do, n't, know, about, everyone, else, ,, b..."
3,1412014244,2ht4oe,41.0,"So, I was browsing Askreddit and found ""Someth...",StoriesAboutKevin,Just found out about this today,en,en,Non-creepy,"(So, ,, I, was, browsing, Askreddit, and, foun..."
4,1414689096,2ksqpi,68.0,I was living with my older brother while atten...,StoriesAboutKevin,I am embarrassed to say that this Kevin is my ...,en,en,Non-creepy,"(I, was, living, with, my, older, brother, whi..."
...,...,...,...,...,...,...,...,...,...,...
495,1251844231,9gd34,2.0,This is all just a thought. I'm not trying to...,self,I think I figured out why the south dominates ...,en,en,Non-creepy,"(This, is, all, just, a, thought, ., , I, 'm,..."
496,1251844989,9gd6a,0.0,If you see a particularly funny/insightful com...,self,Do you downvote comments relative to the ones ...,en,en,Non-creepy,"(If, you, see, a, particularly, funny, /, insi..."
497,1251851288,9gdyq,0.0,"""Love it or leave it."" Do you realize how ret...",self,"Hey Neocons, love it or leave it.",en,en,Non-creepy,"("", Love, it, or, leave, it, ., "", , Do, you,..."
498,1251852724,9ge4y,0.0,Does anyone remember these toy robots that tra...,self,pepsi can robots?,ca,en,Non-creepy,"(Does, anyone, remember, these, toy, robots, t..."


# Visualizing Empath topics and categories

In [41]:
feat_builder = st.FeatsFromOnlyEmpath()
empath_corpus = st.CorpusFromParsedDocuments(convention_df,
                                             category_col='category',
                                             feats_from_spacy_doc=feat_builder,
                                             parsed_col='parse').build()
html = st.produce_scattertext_explorer(empath_corpus,
                                       category='Creepy',
                                       category_name='Creepy',
                                       not_category_name='Non-creepy',
                                       width_in_pixels=1000,
                                       use_non_text_features=True,
                                       use_full_doc=True,
                                       topic_model_term_lists=feat_builder.get_top_model_term_lists())
open("Download/test.html", 'wb').write(html.encode('utf-8'))

9265902

# Visualizing General Inquirer Tag Categories and Document Categories

In [42]:
general_inquirer_feature_builder = st.FeatsFromGeneralInquirer()
corpus = st.CorpusFromPandas(convention_df,
                             category_col='category',
                             text_col='selftext',
                             nlp=st.whitespace_nlp_with_sentences,
                             feats_from_spacy_doc=general_inquirer_feature_builder).build()

/home/anthony/.local/share/virtualenvs/Research-Mapping-Uncanny-Valley-DSc8QBrC/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3437: DtypeWarning: Columns (63,108,109,110,176) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [43]:
html = st.produce_frequency_explorer(corpus,
                                     category='Creepy',
                                     category_name='Creepy',
                                     not_category_name='Non-reepy',
                                     use_non_text_features=True,
                                     use_full_doc=True,
                                     term_scorer=st.LogOddsRatioUninformativeDirichletPrior(),
                                     grey_threshold=1.96,
                                     width_in_pixels=1000,
                                     topic_model_term_lists=general_inquirer_feature_builder.get_top_model_term_lists(),
                                     metadata_descriptions=general_inquirer_feature_builder.get_definitions())

In [44]:
open("Download/test.html", 'wb').write(html.encode('utf-8'))

12128780

[GI Tags](http://www.wjh.harvard.edu/~inquirer/homecat.htm)

# Ordering Terms by Corpus Characteristicness

In [45]:
corpus = (st.CorpusFromPandas(df,
                              category_col='category',
                              text_col='selftext',
                              nlp=st.whitespace_nlp_with_sentences)
          .build()
          .get_unigram_corpus()
          .compact(st.ClassPercentageCompactor(term_count=2,
                                               term_ranker=st.OncePerDocFrequencyRanker)))
html = st.produce_characteristic_explorer(
	corpus,
	category='Creepy',
	category_name='Creepy',
	not_category_name='Non-creepy'
)

In [46]:
open('Download/test.html', 'wb').write(html.encode('utf-8'))

9984458

# Using Cohen's d or Hedge's r to visualize effect size.

In [54]:
convention_df

,party,text,speaker
0,democrat,Thank you. Thank you. Thank you. Thank you so ...,BARACK OBAMA
1,democrat,"Thank you so much. Tonight, I am so thrilled a...",MICHELLE OBAMA
2,democrat,Thank you. It is a singular honor to be here t...,RICHARD DURBIN
3,democrat,"Hey, Delaware. \nAnd my favorite Democrat, Jil...",JOSEPH BIDEN
4,democrat,"Hello. \nThank you, Angie. I'm so proud of how...",JILL BIDEN
...,...,...,...
184,republican,"As the elected leader of 250,000 College Repub...",ALEX SCHRIVER
185,republican,"Good afternoon. I'm Pete Sessions, a congressm...",PETE SESSIONS
186,republican,To Chairman Priebus and to my fellow Americans...,BOB BUCKHORN
187,republican,"\nAbsolutely. Thank you, Mr.Chairman.\nWelcome...",SHARON DAY


In [59]:
corpus = (st.CorpusFromPandas(convention_df,
                              category_col='category',
                              text_col='selftext',
                              nlp=st.whitespace_nlp_with_sentences)
          .build()
          .get_unigram_corpus())

In [60]:
term_scorer = st.CohensD(corpus).set_categories('Creepy', ['Non-creepy'])
term_scorer.get_score_df().sort_values(by='cohens_d', ascending=False).head()

,cohens_d,cohens_d_se,cohens_d_z,cohens_d_p,hedges_r,hedges_r_se,hedges_r_z,hedges_r_p,m1,m2,count1,count2,docs1,docs2
the,0.594362,0.032313,18.393660,7.383301e-76,0.594251,0.033976,17.490487,8.465256e-69,0.049922,0.032908,31740.0,21689.0,1371,2115
eyes,0.484317,0.032083,15.095746,8.636184e-52,0.484226,0.033536,14.438886,1.472817e-47,0.001689,0.000178,957.0,107.0,456,91
room,0.402880,0.031942,12.612885,8.965389e-37,0.402805,0.033266,12.108610,4.754605e-34,0.002244,0.000542,1500.0,432.0,468,238
door,0.385648,0.031915,12.083471,6.457484e-34,0.385576,0.033215,11.608528,1.864914e-31,0.002499,0.000356,1294.0,315.0,413,165
bed,0.369389,0.031891,11.582757,2.519941e-31,0.369320,0.033169,11.134601,4.256264e-29,0.001975,0.000325,831.0,171.0,379,124


In [63]:
html = st.produce_frequency_explorer(
    corpus,
    category='Creepy',
    category_name='Creepy',
    not_category_name='Non-creepy',
    term_scorer=st.CohensD(corpus),
    grey_threshold=0
)

In [64]:
open('Download/test.html', 'wb').write(html.encode('utf-8'))

8602314